In [1]:
!pip install --upgrade --user --quiet google-cloud-aiplatform google-cloud-storage


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import pandas as pd
import vertexai
import tqdm
import time
from vertexai.preview.language_models import TextEmbeddingModel
from google.cloud import storage, aiplatform

In [3]:
data_frame = pd.read_csv('/content/dataResults.csv')

In [4]:
data_frame['id'] = range(1, len(data_frame) + 1)

In [5]:
data_frame.head()

,video_str,play_id,title,ExitVelocity,HitDistance,LaunchAngle,video,id
0,John Jaso hits a home run for the Pittsburgh P...,5b254850-9e14-48d2-8baf-c3e12ecbe68d,John Jaso homers (6) on a fly ball to center f...,102.9,412.0,25.0,https://sporty-clips.mlb.com/YjlLTlpfWGw0TUFRP...,1
1,A baseball player pitches the ball to a batter...,0110e760-5d49-4c53-ad29-e74c787c038d,Evan Longoria homers (34) on a fly ball to lef...,103.1,423.0,31.0,https://sporty-clips.mlb.com/OHcxUXlfWGw0TUFRP...,2
2,"Sure, here's the summary of the video:\n\nA ba...",3b9ecf94-e3fe-4d79-8b33-efe174dfa53e,Marwin Gonzalez homers (13) on a line drive to...,106.6,390.0,22.0,https://sporty-clips.mlb.com/Mk9vbFBfWGw0TUFRP...,3
3,The batter hits the ball deep into right field...,bda1cb83-dca9-46b2-aaad-6c1f009cd51c,Eddie Rosario homers (10) on a fly ball to rig...,98.5,368.0,30.0,https://sporty-clips.mlb.com/ajlhWFpfWGw0TUFRP...,4
4,"Darvish throws a pitch to Semien, who hits the...",9f87e907-3c4f-4187-8fb0-d6784b3b1f1a,Marcus Semien homers (26) on a fly ball to lef...,106.0,435.0,32.0,https://sporty-clips.mlb.com/bGUyNEdfWGw0TUFRP...,5


In [6]:
model = TextEmbeddingModel.from_pretrained("text-embedding-005")

In [7]:
def get_embeddings_wrapper(texts, batch_size=5):
    embeddings = []
    for i in tqdm.tqdm(range(0, len(texts), batch_size)):
        time.sleep(12)
        batch_texts = texts[i:i+batch_size]
        batch_embeddings = model.get_embeddings(batch_texts)
        embeddings.extend([embedding.values for embedding in batch_embeddings])
    return embeddings

combined_texts = data_frame['video_str'].tolist()
data_frame['embedding'] = get_embeddings_wrapper(combined_texts)


100%|██████████| 20/20 [04:02<00:00, 12.14s/it]


In [8]:
data_frame.head()

,video_str,play_id,title,ExitVelocity,HitDistance,LaunchAngle,video,id,embedding
0,John Jaso hits a home run for the Pittsburgh P...,5b254850-9e14-48d2-8baf-c3e12ecbe68d,John Jaso homers (6) on a fly ball to center f...,102.9,412.0,25.0,https://sporty-clips.mlb.com/YjlLTlpfWGw0TUFRP...,1,"[-0.012194664217531681, 0.1299474686384201, 0...."
1,A baseball player pitches the ball to a batter...,0110e760-5d49-4c53-ad29-e74c787c038d,Evan Longoria homers (34) on a fly ball to lef...,103.1,423.0,31.0,https://sporty-clips.mlb.com/OHcxUXlfWGw0TUFRP...,2,"[-0.010519739240407944, 0.12226243317127228, -..."
2,"Sure, here's the summary of the video:\n\nA ba...",3b9ecf94-e3fe-4d79-8b33-efe174dfa53e,Marwin Gonzalez homers (13) on a line drive to...,106.6,390.0,22.0,https://sporty-clips.mlb.com/Mk9vbFBfWGw0TUFRP...,3,"[0.0008175185066647828, 0.06740811467170715, 0..."
3,The batter hits the ball deep into right field...,bda1cb83-dca9-46b2-aaad-6c1f009cd51c,Eddie Rosario homers (10) on a fly ball to rig...,98.5,368.0,30.0,https://sporty-clips.mlb.com/ajlhWFpfWGw0TUFRP...,4,"[-0.014278078451752663, 0.09671445190906525, -..."
4,"Darvish throws a pitch to Semien, who hits the...",9f87e907-3c4f-4187-8fb0-d6784b3b1f1a,Marcus Semien homers (26) on a fly ball to lef...,106.0,435.0,32.0,https://sporty-clips.mlb.com/bGUyNEdfWGw0TUFRP...,5,"[0.02905752696096897, 0.04107468202710152, -0...."


In [9]:
jsonl_string = data_frame[["id", "play_id","ExitVelocity","HitDistance","LaunchAngle","title","video","video_str","embedding"]].to_json(orient="records", lines=True)

with open("candidates.json", "w") as f:
    f.write(jsonl_string)

In [13]:
BUCKET_URI = f"bucket uri"
! gsutil cp candidates.json {BUCKET_URI}

Copying file://candidates.json [Content-Type=application/json]...
/ [1 files][  1.0 MiB/  1.0 MiB]                                                
Operation completed over 1 objects/1.0 MiB.                                      


In [11]:
from google.cloud import aiplatform

vertexai.init(project="project ID" , location="Project Location")

In [14]:
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=f"display_name",
    contents_delta_uri=BUCKET_URI,
    dimensions=768,
    approximate_neighbors_count=5,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Creating MatchingEngineIndex
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:Create MatchingEngineIndex backing LRO: projects/116334440043/locations/us-central1/indexes/4759385614409793536/operations/3263695654811074560
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:MatchingEngineIndex created. Resource name: projects/116334440043/locations/us-central1/indexes/4759385614409793536
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:To use this MatchingEngineIndex in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index:index = aiplatform.MatchingEngineIndex('projects/116334440043/locations/us-central1/indexes/4759385614409793536')


In [15]:
my_index_id = "index id"
my_index = aiplatform.MatchingEngineIndex(my_index_id)

In [16]:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=f"endpoint name",
    public_endpoint_enabled=True
)




INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Creating MatchingEngineIndexEndpoint
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Create MatchingEngineIndexEndpoint backing LRO: projects/116334440043/locations/us-central1/indexEndpoints/381359011024338944/operations/3389796444377448448
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint created. Resource name: projects/116334440043/locations/us-central1/indexEndpoints/381359011024338944
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:To use this MatchingEngineIndexEndpoint in another session:
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/116334440043/locations/us-central1/indexEndpoints/381359011024338944')


In [17]:
DEPLOYED_INDEX_ID = f"dep_index"
index_endpoint.deploy_index(index=my_index, deployed_index_id=DEPLOYED_INDEX_ID,min_replica_count=1)

INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/116334440043/locations/us-central1/indexEndpoints/381359011024338944
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/116334440043/locations/us-central1/indexEndpoints/381359011024338944/operations/7343956917208743936
INFO:google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint:MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/116334440043/locations/us-central1/indexEndpoints/381359011024338944


resource name: projects/116334440043/locations/us-central1/indexEndpoints/381359011024338944

In [19]:
user_prompt = "I am looking for exiting match"
test_embeddings = get_embeddings_wrapper([user_prompt])

new_index_endpoint = aiplatform.MatchingEngineIndexEndpoint("index enpoint")
response = index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=5,
)

100%|██████████| 1/1 [00:12<00:00, 12.12s/it]


In [20]:
import numpy as np

for idx, neighbor in enumerate(response[0]):
    id = np.int64(neighbor.id)
    similar = data_frame.query("id == @id", engine="python")
    print(f"{neighbor.distance:.4f} {similar.title.values[0]} {similar.video.values[0]} {similar.ExitVelocity.values[0]}-{similar.HitDistance.values[0]}-{similar.LaunchAngle.values[0]}")

0.4555 Joc Pederson homers (25) on a fly ball to right field.    Andrew Toles scores. https://sporty-clips.mlb.com/RzY4OUxfWGw0TUFRPT1fQVZVREJWVlZVbEVBQzFOWFVRQUFCZ0ZVQUFOUkJsTUFBRkVHQ1FvRENWQlhBZ3BX.mp4 106.8-398.0-23.0
0.4354 Pedro Alvarez homers (22) on a fly ball to right field. https://sporty-clips.mlb.com/ekw2NVdfWGw0TUFRPT1fQmdoVkJsQU5CVmNBRGxGV1h3QUFCZ1JlQUFCUlZGRUFDZ1lCQWdJTkIxY0dCbEZT.mp4 101.8-405.0-28.0
0.4334 Miguel Montero homers (7) on a fly ball to left field. https://sporty-clips.mlb.com/N1JEamJfWGw0TUFRPT1fVUFNRFZRRUFCRllBWEZNSFZ3QUFBZ1FEQUFOVFcxa0FWRkFBVWxJSFVGWlZWUVVF.mp4 97.9-364.0-35.0
0.4266 Byron Buxton homers (8) on a fly ball to right center field. https://sporty-clips.mlb.com/WDJ4NWtfWGw0TUFRPT1fQXdWUUFWQU1Wd1FBRGxGWFZBQUFVQUJXQUFBTVVBQUFWMXdDQWdCVUJBZFdCd2Rl.mp4 104.4-418.0-28.0
0.4230 David Ortiz homers (37) on a fly ball to right field.    Xander Bogaerts scores. https://sporty-clips.mlb.com/UGdiZWRfWGw0TUFRPT1fQlFaVVhBRUFWd3NBWEFFTFVRQUFWRlZlQUZrQ1VWRUFBV